In [1]:
'''Author Arnab Debnath
Task 4b code
This code takes the Squad dataset. Trasnlate context and the answer to some random langauge. 
Translate the questions to some other random language. Any of the chosen random language can also be English.
'''

'Author Arnab Debnath\nTask 4b code\nThis code takes the Squad dataset. Trasnlate context and the answer to some random langauge. \nTranslate the questions to some other random language.\n'

In [2]:
import os
import json
import copy
import random

In [3]:
random.seed(123)

In [4]:
from google_trans_new import google_translator

In [5]:
SQUAD_FILE_PATH = '../Assignment-4/dataset/squad_train-v1.1.json'

In [6]:
squad_dataset = json.load(open(SQUAD_FILE_PATH))

In [7]:
translator = google_translator(timeout=10)
SRC_LANG = 'en'
target_languages = ['ar', 'bn', 'en', 'fi', 'id', 'ko', 'ru', 'sw', 'te']
context_answer_count = [0,0,0,0,0,0,0,0,0]
question_count = [0,0,0,0,0,0,0,0,0]

In [ ]:
squad_dataset_copy = copy.deepcopy(squad_dataset)
data = squad_dataset_copy['data']

In [ ]:
for index, i in enumerate(data):
    title = i['title']
    paragraphs = i['paragraphs']
    k = 0
    while k < len(paragraphs):
        paragraph = paragraphs[k]
        context = paragraph['context']
        
        context_answer_random_no = random.randint(0, len(target_languages)-1)
        context_answer_random_language = target_languages[context_answer_random_no]
        context_answer_count[context_answer_random_no] += 1
        try:
            if context_answer_random_language != 'en':
                translated_context = translator.translate(
                    context, 
                    lang_src=SRC_LANG, 
                    lang_tgt=context_answer_random_language)
            else:
                translated_context = context
        except:
            # Sometimes this happens for no reason. Translating it again solves it.
            print("context")
            translated_context = translator.translate(context, 
                                                      lang_src=SRC_LANG, 
                                                      lang_tgt=context_answer_random_language)

        qas = paragraph['qas']
        j = 0
        flag_qas = False
        while j < len(qas):
            qa = qas[j]
            answer = qa['answers'][0] #For each question, there is only 1 answer.
            question = qa['question']
            try:
                while True:
                    question_random_n0 = random.randint(0, len(target_languages)-1)
                    if question_random_n0 != context_answer_random_no:
                        break
                        
                question_random_language = target_languages[question_random_n0]
                question_count['question_random_n0'] += 1
                if question_random_language != 'en':
                    translated_question = translator.translate(question, 
                                                               lang_src=SRC_LANG, 
                                                               lang_tgt=question_random_language)
                else:
                    translated_question = question
            except:
                # Sometimes this happens for no reason. Translating it again solves it.
                print("Question", question)
                translated_question = translator.translate(question,
                                                           lang_src=SRC_LANG, 
                                                           lang_tgt=question_random_language)
            answer_start = answer['answer_start']

            text = answer['text']
            try:
                if context_answer_random_language != 'en':
                    translated_text = translator.translate(text, 
                                                           lang_src=SRC_LANG,
                                                           lang_tgt=context_answer_random_language)
                else:
                    translated_text = text
            except:
                # Sometimes this happens for no reason. Translating it again solves it.
                print("text", text)
                translated_text = translator.translate(text, 
                                                       lang_src=SRC_LANG, 
                                                       lang_tgt=context_answer_random_language)


            start_index = translated_context.find(translated_text)
            if start_index != -1:
                answer['answer_start'] = start_index
                answer['text'] = translated_text
                qa['question'] = translated_question  

                flag_qas = True
                j += 1
            else:
                qas.pop(j)

        if flag_qas == False:
            paragraphs.pop(k)
        else:
            paragraph['context'] = translated_context
            k += 1
    print(index)
    if len(paragraphs) == 0:
        data.pop(index)
        print("ALL PARAGRAPHS REMOVED.")

In [ ]:
out_path = '../Assignment-4/dataset/'

In [ ]:
out_file = 'squad_multipleLanguageContext,Question,Answer.json'
out_file_path = os.path.join(out_path, out_file)
with open(out_file_path, 'w', encoding='utf8') as outfile:
    json.dump(squad_dataset_copy, outfile)

In [ ]:
print(context_answer_count)
print(question_count)